<a href="https://colab.research.google.com/github/HB-enpitsu/garakuta/blob/main/%E3%83%AC%E3%83%B3%E3%82%BF%E3%83%AB%E3%83%87%E3%83%BC%E3%82%BF%E3%83%99%E3%83%BC%E3%82%B9%E6%93%8D%E4%BD%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [196]:
import sqlite3

In [197]:
import pandas as pd

In [198]:
DATABASE_FILE = "myrental.db"

# データベース操作

In [199]:
def temp_table():

    return

    # データベース読み込み
    conn = sqlite3.connect(DATABASE_FILE)
    conn.row_factory = sqlite3.Row
    c = conn.cursor()

    c.execute("CREATE TEMPORARY TABLE temp_table AS SELECT * FROM rentals INNER JOIN items ON rentals.item_id = items.id INNER JOIN users ON rentals.user_id = users.id")

    # データベースを閉じる
    c.close()

temp_table()

In [200]:
def all_view():

    # データベース読み込み
    conn = sqlite3.connect(DATABASE_FILE)
    conn.row_factory = sqlite3.Row
    c = conn.cursor()

    c.execute("SELECT * FROM rentals INNER JOIN items ON rentals.item_id = items.id INNER JOIN users ON rentals.user_id = users.id")

    result = [dict(row) for row in c.fetchall()]

    return pd.DataFrame(result)

all_view()

,id,user_id,item_id,start_time,out_count,end_time,in_count,cost,item_name,base,price,total,stock,user_name,tel
0,1,1,1,2023-08-01 17:00:00,1,2023-08-02 09:00:00,1.0,1500,テント（4人用）,0,1500,10,10,田中 太郎,080-1234-5678
1,2,2,2,2023-08-01 10:00:00,1,2023-08-01 15:00:00,1.0,2000,キャンプ用バーベキューセット,0,2000,8,8,佐藤 花子,090-9876-5432
2,3,3,3,2023-08-01 10:00:00,1,2023-08-01 15:00:00,2.0,1600,テーブル＆チェアセット,0,800,12,13,鈴木 一郎,070-4321-8765
3,4,1,4,2023-08-01 18:00:00,2,2023-08-02 09:00:00,1.0,500,キャンプ用ランタン,0,500,20,19,田中 太郎,080-1234-5678
4,5,2,5,2023-08-01 20:00:00,1,None,NaN,700,寝袋,0,700,15,15,佐藤 花子,090-9876-5432


# 在庫確認

In [201]:
# 在庫数再計算

def stock_recal():

    # データベース読み込み
    conn = sqlite3.connect(DATABASE_FILE)
    c = conn.cursor()

    # レンタル履歴
    c.execute("SELECT item_id, SUM(out_count - in_count) AS count FROM rentals WHERE out_count <> in_count GROUP BY item_id")

    lending_counts = c.fetchall()

    # 在庫を更新
    for item_id, count in lending_counts:
        c.execute("UPDATE items SET stock = total - ? WHERE id = ?", (count, item_id))

    # データベースに書き込み
    conn.commit()

    # データベースを閉じる
    c.close()

stock_recal()

# レンタル品確認

In [202]:
# レンタル品確認

def view_items():

    # データベース読み込み
    conn = sqlite3.connect(DATABASE_FILE)
    conn.row_factory = sqlite3.Row
    c = conn.cursor()

    c.execute("SELECT * FROM items")

    result = [dict(row) for row in c.fetchall()]

    # データベースを閉じる
    c.close()

    return pd.DataFrame(result)

view_items()

,id,item_name,base,price,total,stock
0,1,テント（4人用）,0,1500,10,10.0
1,2,キャンプ用バーベキューセット,0,2000,8,8.0
2,3,テーブル＆チェアセット,0,800,12,13.0
3,4,キャンプ用ランタン,0,500,20,19.0
4,5,寝袋,0,700,15,15.0
5,6,ドッグラン,1000,100,1,NaN


# レンタル履歴確認

In [203]:
# レンタル履歴確認

def view_rentals():

    stock_recal()

    # データベース読み込み
    conn = sqlite3.connect(DATABASE_FILE)
    conn.row_factory = sqlite3.Row
    c = conn.cursor()

    c.execute("SELECT * FROM rentals")

    result = [dict(row) for row in c.fetchall()]

    # データベースを閉じる
    c.close()

    return pd.DataFrame(result)

view_rentals()

,id,user_id,item_id,start_time,out_count,end_time,in_count,cost
0,1,1,1,2023-08-01 17:00:00,1,2023-08-02 09:00:00,1.0,1500
1,2,2,2,2023-08-01 10:00:00,1,2023-08-01 15:00:00,1.0,2000
2,3,3,3,2023-08-01 10:00:00,1,2023-08-01 15:00:00,2.0,1600
3,4,1,4,2023-08-01 18:00:00,2,2023-08-02 09:00:00,1.0,500
4,5,2,5,2023-08-01 20:00:00,1,None,NaN,700


# ユーザー情報確認

In [204]:
# ユーザー情報確認

def view_users():

    # データベース読み込み
    conn = sqlite3.connect(DATABASE_FILE)
    conn.row_factory = sqlite3.Row
    c = conn.cursor()

    c.execute("SELECT * FROM users")

    result = [dict(row) for row in c.fetchall()]

    # データベースを閉じる
    c.close()

    return pd.DataFrame(result)

view_users()

,id,user_name,tel
0,1,田中 太郎,080-1234-5678
1,2,佐藤 花子,090-9876-5432
2,3,鈴木 一郎,070-4321-8765
